# Influence Function 線形回帰での例

参考：[Influence Functionでインスタンスの重要度を解釈する - Dropout](https://dropout009.hatenablog.com/entry/2021/07/19/223929)

$$
\begin{align}
L(z, \theta) &= (y - x^\top \theta)^2\\
\nabla_{\theta} L(z, \theta) &= - 2 (y - x^\top \theta) x = -2xy + 2 (x^\top \theta) x\\
R(\theta) &= (y - X\theta)^\top (y - X\theta) \\
\nabla_{\theta} R(\theta) &= -2 X^\top y + 2 (X^\top X) \theta \\
H = \nabla_{\theta}^2 R(\theta) &= 2 (X^\top X)\\
\hat{\theta} &= (X^\top X)^{-1} X^\top y
\end{align}
$$

なので
$$
\begin{align}
\mathcal{I}_{up, params}(z)
    &= -H^{-1}_{\hat{\theta}} \nabla_{\theta} L(z, \hat{\theta})\\
    &= -( 2 X^\top X )^{-1} ( - 2 (y - x^\top \hat\theta) x )
\\
\mathcal{I}_{up, loss}(z, z_{test})
    &= - \nabla_{\theta} L(z_{test}, \hat{\theta})^\top H^{-1}_{\hat{\theta}} \nabla_{\theta} L(z, \hat{\theta}) \\
    &= (2 (y_{test} - x_{test}^\top \hat\theta) x_{test} )^\top ( 2X^\top X )^{-1} ( - 2 (y - x^\top \hat\theta) x )
\end{align}
$$


In [123]:
def influence_params(x, y, X, theta):
    hessian = 2 * (X.T @ X)
    nabla_l_train = - 2 * (y - x.T @ theta) * x
    return np.linalg.inv(hessian) @ nabla_l_train

In [122]:
def influence_loss(x, y, x_test, y_test, X, theta):
    nabla_l_test = (y_test - x_test.T @ theta) * x_test
    hessian = 2 * (X.T @ X)
    nabla_l_train = - 2 * (y - x.T @ theta) * x
    return nabla_l_test.T @ np.linalg.inv(hessian) @ nabla_l_train

In [100]:
import numpy as np

# generate data
n, p = 10, 2
np.random.seed(0)
X = np.random.uniform(size=(n, p))
theta = np.random.uniform(size=p).round(1)
print(theta)
e = np.random.normal(scale=0.1, size=n)
y = X @ theta + e

[1.  0.8]


In [101]:
X

array([[0.5488135 , 0.71518937],
       [0.60276338, 0.54488318],
       [0.4236548 , 0.64589411],
       [0.43758721, 0.891773  ],
       [0.96366276, 0.38344152],
       [0.79172504, 0.52889492],
       [0.56804456, 0.92559664],
       [0.07103606, 0.0871293 ],
       [0.0202184 , 0.83261985],
       [0.77815675, 0.87001215]])

In [102]:
class LinearRegression:

    def fit(self, X, y):
        self.theta_ = np.linalg.inv(X.T @ X) @ (X.T @ y)
        return self

    def predict(self, X):
        return X @ self.theta_


In [103]:
# データ全件でのモデル
theta_hat = LinearRegression().fit(X, y).theta_
theta_hat

array([0.73266182, 1.01441283])

In [104]:
# LOO

diffs = np.array([])
theta_wo_z = np.array([])
for i in range(n):
    X_wo_z = np.concatenate((X[:i, ], X[(i+1):, ]), axis=0)
    y_wo_z = np.concatenate((y[:i], y[(i+1):]), axis=0)
    assert X_wo_z.shape[0] == n - 1
    assert y_wo_z.shape[0] == n - 1
    theta_wo_z_i = LinearRegression().fit(X_wo_z, y_wo_z).theta_
    diff_i = np.linalg.norm(theta_wo_z_i - theta_hat)
    diffs = np.append(diffs, diff_i)
    theta_wo_z = np.concatenate([theta_wo_z, theta_wo_z_i])

theta_wo_z = theta_wo_z.reshape(n, -1)

In [125]:
# LOO (params)

diffs = np.array([])
for i in range(n):
    X_wo_z = np.concatenate((X[:i, ], X[(i+1):, ]), axis=0)
    y_wo_z = np.concatenate((y[:i], y[(i+1):]), axis=0)
    assert X_wo_z.shape[0] == n - 1
    assert y_wo_z.shape[0] == n - 1
    theta_wo_z_i = LinearRegression().fit(X_wo_z, y_wo_z).theta_
    diff_i = np.linalg.norm(theta_wo_z_i - theta_hat)  # ノルムがいいのかはわからんがL2ノルムにしてみる
    diffs = np.append(diffs, diff_i)

In [106]:
diffs

array([0.02083284, 0.00520998, 0.00109041, 0.07302494, 0.1084997 ,
       0.0851947 , 0.0131505 , 0.00110383, 0.07352071, 0.02486394])

In [107]:
idxs = np.argsort(diffs)[::-1]
X[idxs]

array([[0.96366276, 0.38344152],
       [0.79172504, 0.52889492],
       [0.0202184 , 0.83261985],
       [0.43758721, 0.891773  ],
       [0.77815675, 0.87001215],
       [0.5488135 , 0.71518937],
       [0.56804456, 0.92559664],
       [0.60276338, 0.54488318],
       [0.07103606, 0.0871293 ],
       [0.4236548 , 0.64589411]])

In [124]:
influence_params(x=X[i], y=y[i], X=X, theta=theta_hat)

array([0.01773356, 0.00998758])

In [ ]:
# influence (params)

diffs = np.array([])
for i in range(n):
    values = influence_params(x=X[i], y=y[i], X=X, theta=theta_hat)
    diff_i = np.linalg.norm(values)  # ノルムがいいのかはわからんがL2ノルムにしてみる
    diffs = np.append(diffs, diff_i)

idxs = np.argsort(diffs)[::-1]
X[idxs]

array([[0.79172504, 0.52889492],
       [0.43758721, 0.891773  ],
       [0.96366276, 0.38344152],
       [0.0202184 , 0.83261985],
       [0.77815675, 0.87001215],
       [0.5488135 , 0.71518937],
       [0.56804456, 0.92559664],
       [0.60276338, 0.54488318],
       [0.07103606, 0.0871293 ],
       [0.4236548 , 0.64589411]])